
# XGBoost调参完全指南
***
[原文链接](https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/ "Complete Guide to Parameter Tuning in XGBoost")

## 简介
***
如果在预测时, 结果不是你预期的样子, 那么使用 XGBoost 吧! XGBoost 算法已经成为许多数据科学家的终极武器. 这是一个高度复杂的算法, 强大到足以处理各种不规则的数据.

使用 XGBoost 训练一个模型很容易. 但是, 进一步改进模型却很困难(至少我花费了很多时间). 这个算法有很多参数. 为了改进模型, 参数调节是必不可少的. 对于像"哪些参数你应该调节? 为了获得最优结果, 这些参数的理想值是多少?"这样的实际问题很难找到答案. 

这篇文章很适合 XGBoost 新手. 在文章中, 我们将学习**调参的艺术**和一些与 XGBoost 相关的有用信息. 当然, 我们也会在 Python 中实践一下这个算法.

## 目录
***
1. [XGBoost的优点](#1)
2. [XGBoost的参数](#2)
3. [调参实例](#3)

<h2 id="1"> 1. XGBoost的优点 </h2>
***
当我在探究算法的性能和其高精度背后的内在原因时, 我发现了很多优点:

1. **正则化**:
    * [正则化](https://www.analyticsvidhya.com/blog/2015/02/avoid-over-fitting-regularization/)能够减少过拟合, 而标准的 GBM 实现没有与 XGBoost 类似的正则化.
    * XGBoost 也被称为: 带正则化的提升技巧(**regularized boosting technique**).
    
2. **并行处理**:
    * XGBoost 实现了并行计算, 相比于 GBM 更快.
    * 但是等一下, 我们知道 [boosting](https://www.analyticsvidhya.com/blog/2015/11/quick-introduction-boosting-algorithms-machine-learning/) 算法是顺序生成子估计器的, 那么它是如何并行化的呢? 我们知道每一颗树只有在前一颗树生成之后才能被构建,那么是什么原因阻止我们用多核生成树呢? 关于并行化进一步参考[Parallel Gradient Boosting Decision Trees](http://zhanpengfang.github.io/418home.html).
    * XGBoost 支持 Hadoop.
    
3. **高灵活性**:
    * XGBoost 允许用户去自定义目标函数(损失函数)和评价指标.
    * 允许使用者自定义给模型添加了一个全新的维度.
    
4. **处理缺失值**:
    * XGBoost 有一个内建的程序去处理缺失值.
    * 使用者除了提供训练数据之外, 还需要提供一个不同的值作为函数的参数. 在遇到缺失值的节点上, XGBoost 会尝试不同的方法, 然后学会采用何种方法处理缺失值, 这样就能应对将来的缺失情况.
    
5. **剪枝**:
    * GBM 在生成节点时, 如果遇到一次划分得到负的损失(negative loss), 算法就会停止划分. 因此, GBM 更多是一个贪心算法.
    * XGBoost 则是采用先按指定的 `max_depth` 生成树, 然后从叶子节点向根节点剪枝, 剪枝会去掉不是正增益的划分.
    * 另一个优点是: 有时一次负增益划分(比方说-2)后紧跟一次正增益划分(比方说10), GBM 会在遇到负增益时停止划分, 但是 XGBoost 会看到组合增益是8而继续划分.

6. **內建交叉验证**:
    * XGBoost 允许使用者在提升过程的每一次迭代运行交叉验证, 因此, 这让你能够在单次运行后得到准确的最优迭代轮数(子估计器的个数).
    * 不同于 GBM 进行网格搜索(grid-search)只能尝试有限个数的参数.
    
7. **在训练过的模型上继续训练**:
    * 使用者能够在上一次训练模型结果的基础上继续训练模型(warm-start), 该功能在特定应用场景很有用.
    * sklearn 库中实现的 GBM 也有这项功能.
    
官方介绍参考链接:

* [XGBoost Guide - Introduction to Boosted Trees](http://xgboost.readthedocs.io/en/latest/model.html)([已阅](./Introduction to Boosted Trees.ipynb))

<h2 id="2"> 2. XGBoost的参数 </h2>
***
XGBoost 的作者将所有的参数分成为三类:
1. General Parameters: 调节整体功能
2. Booster Parameters: 调节每一个 booster(这里指子估计器)
3. Learning Task Parameters: 调节采用的优化方法

我将会给出和 GBM 的类比, 所以强烈建议阅读一下[这篇文章](https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/)学习 GBM 的基础.

### General Parameters
***
这些参数定义了 XGBoost 的整体功能.

1. **`booster[default=gbtree]`**
    * 选择子估计器的类型, 有两种选择:
        * **`gbtree`**: 基于树的模型
        * **`gblinear`**: 线性模型

2. **`silent[default=0]`**
    * 设置为1时, 静音模式(silent mode)被开启(此时, 运行过程消息不会被打印).
    * 使用默认值通常是一个好的选择, 因为运行过程消息可能会有用.
    
3. **`nthread[default to maximum number of threads available if not set]`**
    * 设置使用的线程数, 用于并行处理.
    * 如果你想同时运行你机器上的所有线程, 采用默认值, 程序会自动检测有几个线程可用.
    
这里还有两个参数你不需要操心他们, 让我们继续 Booster Parameters.

### Booster Parameters
***
虽然, 有两种类型的 booster 可用, 但是我们这里只讨论 **tree booster**, 因为它的表现通常比 **linear booster** 好, 所以后者很少使用.

1. **`eta[default=0.3]`**
    * 与 GBM 中的学习率类似
    * 通过减小该值能够使模型更稳定
    * 最终被使用的典型值是: 0.01-0.2

2. **`min_child_weight[default=1]`**
    * 节点想要继续划分, 所需的实例权重之和的最小值.
    * 这和 GBM 中的 **`min_child_leaf`** 相似, 但是不完全相同. **`min_child_weight`** 指的是实例权重之和的最小值, 而 **`min_child_leaf`** 指的是实例数目的最小值.
    * 该参数用来控制过拟合, 太大的值可能会导致欠拟合, 因此, 需要使用交叉验证调参.
    
3. **`max_depth[default=6]`**
    * 单颗树的最大深度.
    * 该参数用来控制过拟合, 使用交叉验证调参.
    * 典型值: 3-10.
    
4. **`max_leaf_nodes`**
    * 单颗树的叶子节点的最大数.
    * 能够用来替代 **`max_depth`**, 因为二叉树的深度和叶子节点个数有确定的函数关系.
    * 在 GBM 中, 如果定义了该参数, **`max_depth`** 会被忽略.
    
5. **`gamma[default=0]`**
    * 叶子节点进一步划分所需的最小损失衰减.
    * 能够确保算法收敛, 该值根据损失函数的不同而不同.

6. **`max_delta_step[default=0]`**
    * 每棵树权重估计允许的最大步长增量. 如果采用默认值0, 表示没有约束. 如果设置为正数, 则权重更新时步长更加保守.
    * 通常该参数是不需要设置的, 但是在 logistic regression 中, 当数据集类别极度不平衡时可能会有帮助.
    * 该参数通常采用默认值, 但是可以进一步探究.
    
7. **`subsample[default=1]`**
    * 训练每棵树时, 采用的训练数据的比例.
    * 更小的值使算法更加保守, 从而抑制过拟合, 但是太小的值可能导致欠拟合.
    * 典型值: 0.5-1.0.
    
8. **`colsample_bytree[default=1]`**
    * 训练每棵树时, 采用的特征子集的比例.
    * 典型值: 0.5-1.0
    
9. **`colsample_bylevel[default=1]`**
    * 节点划分时, 采用的特征子集的比例.
    * 我通常不会使用这个参数, 因为设置 **`subsample`** 和 **`colsample_bytree`** 本身对该值就有限制. 有待进一步探究.
    
10. **`lambda[default=1]`**
    * L2 正则化项的权重.
    * 尽管很多人不经常使用它, 但是值得被探究用来控制过拟合.
    
11. **`alpha[default=0]`**
    * L1 正则化项的权重.
    * 在特征维度很高时采用, 能够加快运行速度.
    
12. **`scale_pos_weight[default=1]`**
    * 该参数平衡正负权重.
    * 当遇到正负类别不平衡时, 设置一个大于0的值能够帮助加快收敛速度.
    
### Learning Task Parameters
***
这类参数被用来定义优化目标以及每次迭代中应该计算的评价指标. 

1. **`objective[default=reg:linear]`**
    * 该参数用来指定使用的损失函数, 经常使用的有:
        * **`binary:logistic`** - 二分类 logistic regression, 返回的是预测概率(不是类别).
        * **`multi:softmax`** - 使用 softmax 函数的多分类, 返回预测类别(不是概率).
            * 这时需要额外设置一个 **`num_class`** 参数来指定分类类别的个数.
        * **`multi:softprob=`** - 与 **`multi:softmax`** 相同, 但是返回的是预测概率.
        
2. **`eval_metric[default according to objective]`**
    * 在验证集上需要计算的指标.
    * 回归问题默认是 rmse, 分类问题则是 error.
    * 典型值有:
        * **`rmse`** - root mean squared error
        * **`mae`** - mean absolute error
        * **`logloss`** - negative log-likelihood
        * **`error`** - Binary classification error rate(0.5 threshold)
        * **`merror`** - Multiclass classification error rate
        * **`mlogloss`** - Multiclass logloss
        * **`auc`** - Area under the curve
        
3. **`seed[default=0]`**
    * 随机数发生器种子.
    * 能够用来产生可重复的结果, 也能用来调参.
    
如果你一直使用 Scikit-Learn 库, 那么这些参数名可能很陌生, 但是 XGBoost 库提供了 sklearn 命名风格的接口(XGBClassifier). 参数对应关系如下:

1. eta -> learning_rate
2. lambda -> reg_lambda
3. alpha -> reg_alpha

你可能在想 GBM 中的"n_estimators"参数跑哪儿去了, 在 XGBoost 中是"num_boosting_rounds"(在调用 fit 函数时传入).

参考链接:
1. [XGBoost Parameters(official guide)](http://xgboost.readthedocs.io/en/latest/parameter.html#general-parameters)
2. [XGBoost Demo Codes(xgboost GitHub repository)](https://github.com/dmlc/xgboost/tree/master/demo/guide-python)
3. [Python API Reference(official guide)](http://xgboost.readthedocs.io/en/latest/python/python_api.html)

<h2 id="3"> 3. 调参实例 </h2>
***
我们将会使用来自 Data Hackathon 3.x AV 的数据, 问题的详情见[比赛页面](http://datahack.analyticsvidhya.com/contest/data-hackathon-3x). 你能在[这里](https://www.analyticsvidhya.com/wp-content/uploads/2016/02/Dataset.rar)下载数据集. 该数据集在原数据集上进行了预处理.

下面先导入所需的库和加载数据集:

In [19]:
#import libraries:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional sklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

train = pd.read_csv('/home/tyj/Jupyter_ws/xgboost_ex/dataset/Dataset/train_modified.csv')
target = 'Disbursed'
IDcol = 'ID'

Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x7f5e27855390>>
Traceback (most recent call last):
  File "/home/tyj/anaconda3/lib/python3.6/site-packages/xgboost/core.py", line 368, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


In [20]:
train.head()

,Disbursed,Existing_EMI,ID,Loan_Amount_Applied,Loan_Tenure_Applied,Monthly_Income,Var4,Var5,Age,EMI_Loan_Submitted_Missing,...,Var2_2,Var2_3,Var2_4,Var2_5,Var2_6,Mobile_Verified_0,Mobile_Verified_1,Source_0,Source_1,Source_2
0,0.0,0.0,ID000002C20,300000.0,5.0,20000,1,0,37,1,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
1,0.0,0.0,ID000004E40,200000.0,2.0,35000,3,13,30,0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
2,0.0,0.0,ID000007H20,600000.0,4.0,22500,1,0,34,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,0.0,0.0,ID000008I30,1000000.0,5.0,35000,3,10,28,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,0.0,25000.0,ID000009J40,500000.0,2.0,100000,3,17,31,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


注意, 这里导入了 XGBoost 的两种形式:

1. **`xgb`** - 我们将会使用这个库中的 **`cv`** 函数.
2. **`XGBClassifier`** - 这使我们能够使用 sklearn 中可以并行处理的 **`GridSearchCV`** 类.

下面让我们先定义一个能够帮助我们创建 XGBoost 模型和执行交叉验证的函数. 

In [23]:
def modelfit(alg, dtrain, predictors, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Disbursed'], eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % metrics.accuracy_score(dtrain['Disbursed'].values, dtrain_predictions))
    print("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Disbursed'], dtrain_predprob))
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

这个函数与我们在 GBM 中使用的函数稍有不同. 这篇文章的关注点在于涵盖概念而不是编写代码. 如果有看不明白的地方, 不要纠结. 注意: XGBoost 库的 sklearn 接口没有"feature_importances"指标, 但是有执行相同功能的函数 get_fscore().

### 调参的一般方法
***
我们将在这里使用和 GBM 类似的方法, 以下是要执行的步骤:

1. 选择一个相对高的学习率(learning rate). 通常0.1可以工作, 但是对于不同的问题, 学习率可能需设置在0.05-0.3之间. 接下来选择对于该学习率参数的最佳提升轮数. XGBoost 有一个非常有用的函数 "cv" 能够在每轮提升迭代中执行交叉验证, 从而得到所需的子估计器的最佳数目.

2. 调节子估计器参数(**`max_depth`**, **`min_child_weight`**, **`gamma`**, **`subsample`**, **`colsample_bytree`**). 注意: 我们可以挑选不同的参数来定义子估计器(单颗树), 这里仅此一例.

3. 调节正则化项相关的参数(**`lambda`**, **`alpha`**), 能降低模型复杂度, 提高模型性能.

4. 选择更小的学习率参数, 找到最优的那个.

下面让我们看一下这些步骤更详细的介绍.

### Step 1: Fix learning rate and number of estimators for tuning tree-based parameters
***
为了确定提升参数(这里指学习率和子估计器个数), 我们需要给其他参数设定一个初始值, 让我们按照如下设置:

1. **`max_depth = 5`**: 该参数应该在 3-10 之间. 我在这里选择5, 你可以尝试 4-6 之间的值.

2. **`min_child_weight = 1`**: 这里选择一个很小的值, 因为这是一个类别不平衡问题, 这样叶子节点分得的样本会更少.

3. **`gamma = 0`**: 也可选择 0.1-0.2 之间的值作为初始值, 该值将稍后调整.

4. **`subsample, colsample_bytree = 0.8`**: 0.8是一个常用的初始值, 典型取值范围是 0.5-0.9 之间.

5. **`scale_pos_weight = 1`**: 因为训练集类别高度不平衡.

注意: 以上参数只是初始值, 后面会一一调节. 让我们先采用默认的学习率参数0.1, 然后使用 cv 函数找到最佳子估计器个数. 上面定义的函数可以实现这个目的.

In [24]:
#Choose all predictors except target & IDcols
predictors = [x for x in train.columns if x not in [target, IDcol]]
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, train, predictors)


Model Report
Accuracy : 0.9854
AUC Score (Train): 0.891681


TypeError: 'str' object is not callable

你可以看见, 对于0.1的学习率, 最优子估计器数目是140. 注意: 该数可能大了. 如果太大, 你可以增加学习率, 然后重新运行以上代码得到更小的最优子估计器数目.

### Step 2: Tune `max_depth` and `min_child_weight`
***
我们先调这两个参数, 因为这两个参数会对模型结果的影响最大. 我们先设置一个较宽的范围, 然后在慢慢缩小.

**注意**: 这里将要执行计算量很大的网格搜索, 这可能会花费15-30分钟的时间.

In [28]:
param_test1 = {
 'max_depth': list(range(3,10,2)),
 'min_child_weight': list(range(1,6,2))
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(train[predictors],train[target])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.83764, std: 0.00875, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: 0.83837, std: 0.00825, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: 0.83716, std: 0.00818, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: 0.84016, std: 0.00680, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: 0.83965, std: 0.00537, params: {'max_depth': 5, 'min_child_weight': 3},
  mean: 0.83935, std: 0.00548, params: {'max_depth': 5, 'min_child_weight': 5},
  mean: 0.83570, std: 0.00587, params: {'max_depth': 7, 'min_child_weight': 1},
  mean: 0.83448, std: 0.00726, params: {'max_depth': 7, 'min_child_weight': 3},
  mean: 0.83456, std: 0.00554, params: {'max_depth': 7, 'min_child_weight': 5},
  mean: 0.82851, std: 0.00651, params: {'max_depth': 9, 'min_child_weight': 1},
  mean: 0.82955, std: 0.00580, params: {'max_depth': 9, 'min_child_weight': 3},
  mean: 0.83158, std: 0.00677, params: {'max_depth': 9, 'min_child_weight': 5}],
 {'max_depth': 5, 'min_child_weight': 1

这里测试了12种不同的参数组合, 间距为2. 最优值是 **`max_depth=5`**, **`min_child_weight=1`**. 接下来让我们进一步寻找更优的参数.

In [29]:
param_test2 = {
 'max_depth': [4,5,6],
 'min_child_weight': [0.1,1,2]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', scale_pos_weight=1,seed=27), 
 param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(train[predictors],train[target])
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

([mean: 0.84117, std: 0.00688, params: {'max_depth': 4, 'min_child_weight': 0.1},
  mean: 0.84057, std: 0.00670, params: {'max_depth': 4, 'min_child_weight': 1},
  mean: 0.84122, std: 0.00667, params: {'max_depth': 4, 'min_child_weight': 2},
  mean: 0.84016, std: 0.00529, params: {'max_depth': 5, 'min_child_weight': 0.1},
  mean: 0.84016, std: 0.00680, params: {'max_depth': 5, 'min_child_weight': 1},
  mean: 0.83980, std: 0.00602, params: {'max_depth': 5, 'min_child_weight': 2},
  mean: 0.83802, std: 0.00564, params: {'max_depth': 6, 'min_child_weight': 0.1},
  mean: 0.83877, std: 0.00519, params: {'max_depth': 6, 'min_child_weight': 1},
  mean: 0.83896, std: 0.00565, params: {'max_depth': 6, 'min_child_weight': 2}],
 {'max_depth': 4, 'min_child_weight': 2},
 0.8412189672353089)

这里得到最优值是 **`max_depth=4`**, **`min_child_weight=2`**. 可以看见交叉验证得分轻微上升. 注意: 随着模型性能的提升, 提升模型的性能变得非常难. 

In [32]:
param_test2b = {
 'min_child_weight': [2,4,6,8,10]
}
gsearch2b = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=4,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', scale_pos_weight=1,seed=27), 
 param_grid = param_test2b, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2b.fit(train[predictors],train[target])
modelfit(gsearch2b.best_estimator_, train, predictors)
gsearch2b.grid_scores_, gsearch2b.best_params_, gsearch2b.best_score_


Model Report
Accuracy : 0.9854
AUC Score (Train): 0.878673


TypeError: 'str' object is not callable

In [33]:
gsearch2b.grid_scores_, gsearch2b.best_params_, gsearch2b.best_score_

([mean: 0.84122, std: 0.00667, params: {'min_child_weight': 2},
  mean: 0.84034, std: 0.00601, params: {'min_child_weight': 4},
  mean: 0.84003, std: 0.00622, params: {'min_child_weight': 6},
  mean: 0.83889, std: 0.00714, params: {'min_child_weight': 8},
  mean: 0.84004, std: 0.00661, params: {'min_child_weight': 10}],
 {'min_child_weight': 2},
 0.8412189672353089)

### Step 3: Tune **`gamma`**
***
现在开始调节 **`gamma`** 参数, 利用上面已经调好的参数.

In [34]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=4,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(train[predictors],train[target])
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

([mean: 0.84122, std: 0.00667, params: {'gamma': 0.0},
  mean: 0.84042, std: 0.00732, params: {'gamma': 0.1},
  mean: 0.83969, std: 0.00840, params: {'gamma': 0.2},
  mean: 0.84038, std: 0.00707, params: {'gamma': 0.3},
  mean: 0.83951, std: 0.00812, params: {'gamma': 0.4}],
 {'gamma': 0.0},
 0.8412189672353089)

 可以看见, **`gamma`**的最优值是初始值. 在调节其他参数之前, 一个好的选择是重新调节子估计器个数.

In [35]:
xgb2 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=4,
 min_child_weight=2,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb2, train, predictors)


Model Report
Accuracy : 0.9854
AUC Score (Train): 0.887868


TypeError: 'str' object is not callable

最后参数是:
* **`max_depth=4`**
* **`min_child_weight=2`**
* **`gamma=0`**

### Step 4: Tune **`subsample`** 和 **`colsample_bytree`**
***

In [36]:
param_test4 = {
 'subsample': [i/10.0 for i in range(6,10)],
 'colsample_bytree': [i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=4,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', scale_pos_weight=1,seed=27), 
 param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch4.fit(train[predictors],train[target])
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

([mean: 0.83899, std: 0.00943, params: {'colsample_bytree': 0.6, 'subsample': 0.6},
  mean: 0.83678, std: 0.00852, params: {'colsample_bytree': 0.6, 'subsample': 0.7},
  mean: 0.83878, std: 0.00798, params: {'colsample_bytree': 0.6, 'subsample': 0.8},
  mean: 0.83960, std: 0.00763, params: {'colsample_bytree': 0.6, 'subsample': 0.9},
  mean: 0.84048, std: 0.00841, params: {'colsample_bytree': 0.7, 'subsample': 0.6},
  mean: 0.84028, std: 0.00819, params: {'colsample_bytree': 0.7, 'subsample': 0.7},
  mean: 0.83916, std: 0.00805, params: {'colsample_bytree': 0.7, 'subsample': 0.8},
  mean: 0.84015, std: 0.00785, params: {'colsample_bytree': 0.7, 'subsample': 0.9},
  mean: 0.84168, std: 0.00900, params: {'colsample_bytree': 0.8, 'subsample': 0.6},
  mean: 0.83943, std: 0.00877, params: {'colsample_bytree': 0.8, 'subsample': 0.7},
  mean: 0.84136, std: 0.00737, params: {'colsample_bytree': 0.8, 'subsample': 0.8},
  mean: 0.84151, std: 0.00876, params: {'colsample_bytree': 0.8, 'subsample'

In [37]:
param_test5 = {
 'subsample':[i/100.0 for i in range(85,100,5)],
 'colsample_bytree':[i/100.0 for i in range(85,100,5)]
}
gsearch5 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=4,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', scale_pos_weight=1,seed=27), 
 param_grid = param_test5, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch5.fit(train[predictors],train[target])

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=2, missing=None, n_estimators=177,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'subsample': [0.85, 0.9, 0.95], 'colsample_bytree': [0.85, 0.9, 0.95]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [38]:
gsearch5.grid_scores_, gsearch5.best_params_, gsearch5.best_score_

([mean: 0.84046, std: 0.00875, params: {'colsample_bytree': 0.85, 'subsample': 0.85},
  mean: 0.84124, std: 0.01005, params: {'colsample_bytree': 0.85, 'subsample': 0.9},
  mean: 0.84140, std: 0.01040, params: {'colsample_bytree': 0.85, 'subsample': 0.95},
  mean: 0.84132, std: 0.00741, params: {'colsample_bytree': 0.9, 'subsample': 0.85},
  mean: 0.84202, std: 0.00897, params: {'colsample_bytree': 0.9, 'subsample': 0.9},
  mean: 0.84168, std: 0.00801, params: {'colsample_bytree': 0.9, 'subsample': 0.95},
  mean: 0.84137, std: 0.00830, params: {'colsample_bytree': 0.95, 'subsample': 0.85},
  mean: 0.84229, std: 0.00757, params: {'colsample_bytree': 0.95, 'subsample': 0.9},
  mean: 0.84161, std: 0.00951, params: {'colsample_bytree': 0.95, 'subsample': 0.95}],
 {'colsample_bytree': 0.95, 'subsample': 0.9},
 0.8422949410926629)

最后参数是:
* **`subsample=0.9`**
* **`colsample_bytree=0.95`**

### Step 5: Tuning Regularization Parameters
***
接下来应用正则化去控制过拟合. 很多人很少使用该参数, 因为 **`gamma`** 也能控制模型复杂度. 但是, 我们通常应该尝试一下, 这里我会尝试 **`reg_alpha`**, 你可以自己尝试 **`reg_lambda`**.

In [39]:
param_test6 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=4,
 min_child_weight=2, gamma=0, subsample=0.9, colsample_bytree=0.95,
 objective= 'binary:logistic', scale_pos_weight=1,seed=27), 
 param_grid = param_test6, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch6.fit(train[predictors],train[target])
gsearch6.grid_scores_, gsearch6.best_params_, gsearch6.best_score_

([mean: 0.84229, std: 0.00757, params: {'reg_alpha': 1e-05},
  mean: 0.84131, std: 0.00721, params: {'reg_alpha': 0.01},
  mean: 0.84199, std: 0.00751, params: {'reg_alpha': 0.1},
  mean: 0.84186, std: 0.00794, params: {'reg_alpha': 1},
  mean: 0.81361, std: 0.01559, params: {'reg_alpha': 100}],
 {'reg_alpha': 1e-05},
 0.8422948494447787)

In [40]:
param_test7 = {
 'reg_alpha':[0, 1e-6, 5e-6, 1e-5, 5e-5]
}
gsearch7 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=4,
 min_child_weight=2, gamma=0, subsample=0.9, colsample_bytree=0.95,
 objective= 'binary:logistic', scale_pos_weight=1,seed=27), 
 param_grid = param_test7, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch7.fit(train[predictors],train[target])
gsearch7.grid_scores_, gsearch7.best_params_, gsearch7.best_score_

([mean: 0.84229, std: 0.00757, params: {'reg_alpha': 0},
  mean: 0.84229, std: 0.00757, params: {'reg_alpha': 1e-06},
  mean: 0.84229, std: 0.00757, params: {'reg_alpha': 5e-06},
  mean: 0.84229, std: 0.00757, params: {'reg_alpha': 1e-05},
  mean: 0.84229, std: 0.00757, params: {'reg_alpha': 5e-05}],
 {'reg_alpha': 0},
 0.8422949410926629)

In [41]:
xgb3 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=4,
 min_child_weight=2,
 gamma=0,
 subsample=0.9,
 colsample_bytree=0.95,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb3, train, predictors)


Model Report
Accuracy : 0.9854
AUC Score (Train): 0.888192


TypeError: 'str' object is not callable

### Step 6: Reducing Learning Rate
***
最后, 我们应该减小学习率, 这样会得到更多子估计器.

In [42]:
xgb4 = XGBClassifier(
 learning_rate =0.01,
 n_estimators=5000,
 max_depth=4,
 min_child_weight=2,
 gamma=0,
 subsample=0.9,
 colsample_bytree=0.95,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb4, train, predictors)


Model Report
Accuracy : 0.9854
AUC Score (Train): 0.884858


TypeError: 'str' object is not callable

***
这里, 我想分享两个关键点:

1. 仅仅通过调参和使用稍好的模型很难获得很大的性能提升. 
2. 想要更大的性能提升, 可以尝试一下: 特征工程(feature engineering), 模型集成(ensemble of models), stacking 等等.